In [1]:
# -*- coding: utf-8 -*-
"""
Created on Thu Jul  2 15:01:53 2020
@author: Kam Look
"""
import parseTrans
import bondAndComp
import postCompClean

This will only be for rapid testing and viewing dataframes. Real edits and edits to functions should be done in Sublime or text editor of choice

In [2]:
pathCities = r'D:\PPI Matching Names\Cities'
pathSpecDis = r'D:\PPI Matching Names\Special Districts'
pathCounties = r'D:\PPI Matching Names\Counties'
pathBond = r'D:\PPI Matching Names\BondAdaptPeople-Data.csv'

In [3]:
pathConstruct = r'D:\PPI Matching Names\PossibleJobs\construction.csv'
pathEng1 = r'D:\PPI Matching Names\PossibleJobs\engineer1.csv'
pathEngSupp = r'D:\PPI Matching Names\PossibleJobs\Engineering Support.csv'
pathPlanning = r'D:\PPI Matching Names\PossibleJobs\planning.csv'
pathHR = r'D:\PPI Matching Names\PossibleJobs\HR.csv'

In [7]:
paths = [pathPlanning, pathHR, pathConstruct, pathEngSupp, pathEng1] # will be iterated through    
    
    
def main(pathCA, pathBond, paths=None):
    '''
    INPUTS
    str     pathCA: path to directory with Transparent California Files
    str     pathBond: path to Bond-People csv
    list    paths: list of paths bond files with unique job titles
    '''
            
    _, jobsDF, filterTag = parseTrans.parse_transparent_data(pathCA,paths)
    bondDF = bondAndComp.parse_bond_data(pathBond)
    comp_trans_only, comp_shared = bondAndComp.compare_dataframes(bondDF, jobsDF)
    comp_shared['Confidence Level'] = comp_shared.apply(lambda row: postCompClean.confidence_matching(row['Job Title Bond'], row['Job Title'],
                                                                                        row['Bond Company'], row['Agency']),axis=1)
    comp_shared['Action'] = comp_shared.apply(lambda row: postCompClean.define_action(row['Confidence Level']), axis=1)
    #only comment keeping beacuse kinda proud of the below line 
    #comp_shared['Action'] = comp_shared.apply(lambda row:'Keep' if row['Confidence Level'] =='Very High' or row['Confidence Level']=='High' else 'Leave', axis=1)
    pathShared = postCompClean.saving_csv(comp_trans_only, comp_shared, filterTag)
    pathShared = 'D:\PPI Matching Names\SharedPeople\hrSharedBACKUP.csv' # path used for testing that has all manual check removed 
    comp_trans_only, keepDF = postCompClean.manual_edits(pathShared,comp_trans_only) # if i can call this returning final, that would be nice
    
    
    return comp_trans_only, keepDF

In [22]:
'Testing Main'
CTO, CS = main(pathCounties,pathBond)


D:\Anaconda\lib\site-packages\ipykernel_launcher.py:2: DtypeWarning: Columns (2) have mixed types.Specify dtype option on import or set low_memory=False.
  


No path returned, using built-in system values
1: Planners
2: HR
3: Construction
4: Engineering Support
5: Engineering 1
6: No extra filter
Choose extra filter from selection above: 1
Reading Time: 0:00:00.492714
Sorting and Selecting Time: 0:00:02.772409
Total time: 0:00:03.268122
What kind of area are you searching? 
[City, SpecDist, or County]County

Once all manual checks have been addressed save and close the file 
Type and submit "Enter" to continue  enter


In [ ]:
'Cell for identifying dupes'
#May be able to get away with removing dupes at the very very end. 

#CTO=CTO.drop(columns=['_merge','Action'])
#CTO.to_csv('combinedInfo.csv', index=False)
final_dupes = CTO[CTO.duplicated(['First Name','Last Name'],keep=False)]
#CTO
final_dupes.to_csv('FinalDupes_needsChecking.csv',index=False)
final_dupes

In [6]:
'Exporting DataFrames, CTO and CS'
# CTO: All employees ready to be contacted, employees not in bond data base
#CS: Employees we are very confident are in the bond database, don't need to contact them. 
CTO.to_csv('SpecDistCheck_trans.csv',index=False)
CS.to_csv('SpecDistCheck_shared.csv',index=False)